In [32]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import time
from sklearn.cluster import DBSCAN
from typing import List
import re

from tqdm import tqdm
from joblib import Parallel, delayed

In [33]:
df = pd.read_csv('../wypadek.csv')

In [34]:
df.sample(2)

,Data zdarzenia,Godzina,GPS x,GPS y,Miejscowość,Ulica,Numer domu,Numer drogi,KM HM (Pikietaż),Obszar,...,Sygnalizacja świetlna,Oznakowanie poziome,Liczba pasów w zdarzeniu,Zachowanie kierującego,Zachowanie pieszego,Inne przyczyny,Liczba rannych w zdarzeniu,Liczba zabitych w zdarzeniu,Liczba kolizji,Liczba wypadków
172800,2019-01-24,19:00:00,18*31'366,49*57'412,MSZANA,NaN,NaN,A1A,40,Obszar niezabudowany,...,Brak,Jest,1.0,NaN,NaN,Stan jezdni,0,0,1,0
97331,2017-10-12,14:35:00,18*56'459,49*58'271,PSZCZYNA,KOŚCIUSZKI,NaN,G340251S,NaN,Obszar zabudowany,...,Brak,Jest,NaN,Nieprawidłowe skręcanie,NaN,NaN,0,0,1,0


### Usuwanie wierszy w złym formacje danych:

df['y_counter'] = df['GPS y'].apply(lambda x: x.count('*'))
df['x_counter'] = df['GPS x'].apply(lambda x: x.count('*'))
df = df[(df['y_counter'] < 2) & (df['x_counter'] < 2)]
df = df.drop(columns=['y_counter', 'x_counter'])

### Konwersja stopnii, minut (dm) na stopnie dziesiętne (dd):

def dm_to_dd(dm):
    deg, minutes = re.split('[*]', dm)
    minutes = minutes.replace("'", '.')
    minutes = float(re.sub("[.]{2,}", '.', minutes))
    return (float(deg) + float(minutes)/60)

def dm_converter(source_columns: List[str], target_columns: List[str], data_frame: DataFrame):
    for source, target in zip(source_columns, target_columns):
        data_frame[target] = data_frame[source].apply(lambda x: round(dm_to_dd(x), 6))
    return data_frame

df = dm_converter(source_columns=['GPS y', 'GPS x'], target_columns=['Lat', 'Lon'], data_frame=df)
df.head(2)

### Konwersja HH.MM.SS.S -> DD (Michał):

In [35]:
# https://stackoverflow.com/questions/33997361
def dms2dd(coord):
    """ GPS HH.MM.SS.S to DD (Decimal Degrees) conversion """
    c = re.findall(r"\w+", coord)
    degrees = float(c[0])
    minutes = float(c[1])
    if len(c) > 2:
        seconds = float(c[2])/10   
    else:
        seconds = 0
    
    dd = float(degrees) + (minutes)/60 + float(seconds)/(60*60)

    return dd

df['Lat'] = df['GPS y'].apply(lambda x : round(dms2dd(x), 6)) # Latitude - N
df['Lon'] = df['GPS x'].apply(lambda x : round(dms2dd(x), 6)) # Longitude - E

In [36]:
df.head()

,Data zdarzenia,Godzina,GPS x,GPS y,Miejscowość,Ulica,Numer domu,Numer drogi,KM HM (Pikietaż),Obszar,...,Liczba pasów w zdarzeniu,Zachowanie kierującego,Zachowanie pieszego,Inne przyczyny,Liczba rannych w zdarzeniu,Liczba zabitych w zdarzeniu,Liczba kolizji,Liczba wypadków,Lat,Lon
0,2016-01-01,05:15:00,18*38'089,50*22'110,GLIWICE,TOSZECKA,NaN,901,"61,6",Obszar niezabudowany,...,2.0,NaN,NaN,"Obiekty, zwierzęta na drodze",0,0,1,0,50.369722,18.635806
1,2016-01-01,00:05:00,19*10'510,50*19'231,DĄBROWA GÓRNICZA,SIENKIEWICZA,NaN,G200 030S,NaN,Obszar zabudowany,...,NaN,Nieprawidłowe: cofanie,NaN,NaN,0,0,1,0,50.323083,19.180833
2,2016-01-01,18:30:00,18*52'395,50*26'423,TARNOWSKIE GÓRY,KS.IGNACEGO SIWCA,4A,GMINNA,NaN,Obszar zabudowany,...,2.0,Nieprawidłowe: omijanie,NaN,NaN,0,0,1,0,50.445083,18.877639
3,2016-01-01,15:10:00,18*51'381,50*26'408,TARNOWSKIE GÓRY,JÓZEFA PIŁSUDSKIEGO,6,P3276S,NaN,Obszar zabudowany,...,2.0,Nieprawidłowe: omijanie,NaN,NaN,0,0,1,0,50.444667,18.860583
4,2016-01-01,17:45:00,19*02'512,50*28'005,OŻAROWICE,DWORCOWA,16,P3203S,NaN,Obszar zabudowany,...,2.0,NaN,NaN,"Obiekty, zwierzęta na drodze",0,0,1,0,50.466806,19.047556


## GRUPOWANIE DANYCH - algorytm DBSCAN:
https://geoffboeing.com/2014/08/clustering-to-reduce-spatial-data-set-size/

### Obliczenia na jednym wątku:

In [ ]:
def dbscan_lat_lon(data_frame: DataFrame, distance=50):
    start_time = time.time() 
    
    kms_per_radian = 6371.0088
    coords = data_frame.loc[:,['Lat', 'Lon']].values
    epsilon = (distance / 1000) / kms_per_radian
    
    dbscan = DBSCAN(eps=epsilon, 
                    min_samples=1, 
                    algorithm='ball_tree', 
                    metric='haversine')
    
    db = dbscan.fit(np.radians(coords))

    a = pd.DataFrame(data_frame, columns=data_frame.columns)
    a.reset_index(drop=True, inplace=True)
    b = pd.DataFrame(db.labels_, columns=['DBSCAN_Lat_Lon'])
    b.reset_index(drop=True, inplace=True)
   
    
    cluster_labels = db.labels_
    num_start_point = len(data_frame)
    num_clusters = len(set(cluster_labels)) 
    proc_compression = 100 * (1 - float(num_clusters) / num_start_point)
    print(f'Clustered {len(data_frame):,} points down to {num_clusters:,} clusters, '
          f'for {proc_compression:,.2f}% compression.') 
    end_time = time.time()

    print('Calculate time: {:,.2f} seconds'.format(end_time - start_time)) 
    return pd.concat([a,b], axis=1)

In [ ]:
result = dbscan_lat_lon(distance=50, data_frame=df)

### Grupy najczęściej występjące:

In [ ]:
result.groupby(['Miejscowość', 'DBSCAN_Lat_Lon'])['DBSCAN_Lat_Lon'].count().sort_values(ascending=False).head(10)

In [ ]:
result[(result['DBSCAN_Lat_Lon'] == 5539) & (result['Miejscowość'] == 'KATOWICE')].sort_values(by='Lat').head(10)

### Obliczenia wielowątkowe:

In [37]:
distance = 50
chunk = 'Miejscowość'

kms_per_radian = 6371.0088
epsilon = (distance / 1000)/ kms_per_radian


def dbscan_lat_lon(i):
    
    temp_df = df[df[chunk] == i]
    np_array_dict = dataframe_to_arrays(temp_df.columns, temp_df)
    
    dbscan = DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric='haversine')
    db = dbscan.fit(coords_to_radians(df, i))
    return np.c_[np.array(list(np_array_dict.values())).transpose(), db.labels_]

def dataframe_to_arrays(columns: List[str], df):
    np_array_dict = {}
    for j in list(columns):
        np_array_dict[j] = df[j].values.astype('U')
    return np_array_dict

def coords_to_radians(df, i):
    coords = np.radians(df.loc[:, ['Lat', 'Lon']].values [df[chunk] == i])
    return coords

results = Parallel(n_jobs=-1, verbose=10, backend = 'multiprocessing')(delayed(dbscan_lat_lon)(i) for i in tqdm(np.unique(df[chunk].values)))

data_frame_columns_name = list(df.columns)
results = DataFrame(np.concatenate([a for a in results], axis=0),
                    columns=(data_frame_columns_name + list(['DBSCAN_Lat_Lon'])))

  0%|          | 1/1097 [00:00<06:19,  2.89it/s][Parallel(n_jobs=-1)]: Batch computation too fast (0.0802s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.1s
  2%|▏         | 20/1097 [00:00<04:22,  4.10it/s][Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.2s
 39%|███▉      | 432/1097 [00:12<01:02, 10.58it/s][Parallel(n_jobs=-1)]: Batch computation too slow (2.0310s.) Setting batch_size=1.
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:   12.6s
100%|██████████| 1097/1097 [00:32<00:00, 33.56it/s]
[Parallel(n_jobs=-1)]: Done 1097 out of 1097 | elapsed:   33.9s finished


### Grupy najczęściej występjące:

In [39]:
results.head(6)

,Data zdarzenia,Godzina,GPS x,GPS y,Miejscowość,Ulica,Numer domu,Numer drogi,KM HM (Pikietaż),Obszar,...,Zachowanie kierującego,Zachowanie pieszego,Inne przyczyny,Liczba rannych w zdarzeniu,Liczba zabitych w zdarzeniu,Liczba kolizji,Liczba wypadków,Lat,Lon,DBSCAN_Lat_Lon
0,2016-03-14,06:35:00,18*19'234,50*07'494,ADAMOWICE,RYBNICKA,nan,P5604S,nan,Obszar niezabudowany,...,nan,nan,"Obiekty, zwierzęta na drodze",0,0,1,0,50.130389,18.323167,0
1,2016-05-19,18:45:00,18*20'455,50*07'493,ADAMOWICE,RYBNICKA,82,P5604S,nan,Obszar zabudowany,...,Niedostosowanie prędkości do warunków ruchu,nan,nan,0,0,1,0,50.130361,18.345972,1
2,2017-07-07,18:30:00,18*20'474,50*07'491,ADAMOWICE,RYBNICKA,81,P5604S,nan,Obszar niezabudowany,...,nan,nan,Inne,0,0,1,0,50.130306,18.3465,1
3,2019-05-21,15:10:00,18*20'249,50*07'522,ADAMOWICE,RYBNICKA,76B,P5604S,nan,Obszar zabudowany,...,Inne przyczyny,nan,nan,0,0,1,0,50.131167,18.34025,2
4,2019-06-17,10:50:00,18*19'232,50*07'492,ADAMOWICE,RYBNICKA,nan,P5604S,nan,Obszar niezabudowany,...,nan,nan,"Obiekty, zwierzęta na drodze",0,0,1,0,50.130333,18.323111,0
5,2019-09-06,12:40:00,18*21'118,50*07'559,ADAMOWICE,RYBNICKA,nan,P5604S,nan,Obszar niezabudowany,...,Nieprawidłowe omijanie,nan,nan,0,0,1,0,50.132194,18.353278,3


In [42]:
results.groupby(['Miejscowość', 'DBSCAN_Lat_Lon'])['DBSCAN_Lat_Lon'].count().sort_values(ascending=False).head(10)

Miejscowość    DBSCAN_Lat_Lon
KATOWICE       5                 14336
CZĘSTOCHOWA    5                  4301
GLIWICE        8                  2344
BIELSKO-BIAŁA  14                 1860
BYTOM          0                  1499
SOSNOWIEC      2                  1232
ZABRZE         4                   982
CZĘSTOCHOWA    17                  935
               1                   828
RYBNIK         22                  799
Name: DBSCAN_Lat_Lon, dtype: int64

In [40]:
results[(results['DBSCAN_Lat_Lon'] == '5') & (results['Miejscowość'] == 'KATOWICE')].sort_values(by='Lat').head(10)

,Data zdarzenia,Godzina,GPS x,GPS y,Miejscowość,Ulica,Numer domu,Numer drogi,KM HM (Pikietaż),Obszar,...,Zachowanie kierującego,Zachowanie pieszego,Inne przyczyny,Liczba rannych w zdarzeniu,Liczba zabitych w zdarzeniu,Liczba kolizji,Liczba wypadków,Lat,Lon,DBSCAN_Lat_Lon
101723,2017-10-22,16:25:00,19*03'133,50*14'222,KATOWICE,PSZCZYŃSKA,nan,86,nan,Obszar niezabudowany,...,nan,nan,Nieustalone,0,0,1,0,50.2395,19.053694,5
102577,2017-12-01,14:45:00,19*03'103,50*14'228,KATOWICE,GÓRNOŚLĄSKA,nan,G100446S,nan,Obszar niezabudowany,...,Nieprawidłowe zmienianie pasa ruchu,nan,nan,0,0,1,0,50.239667,19.052861,5
117077,2019-12-26,18:00:00,19*03'094,50*14'228,KATOWICE,GÓRNOŚLĄSKA,nan,G100446S,nan,Obszar niezabudowany,...,Niedostosowanie prędkości do warunków ruchu,nan,nan,0,0,1,0,50.239667,19.052611,5
107315,2018-08-26,11:10:00,19*03'084,50*14'229,KATOWICE,MURCKOWSKA,nan,86,nan,Obszar niezabudowany,...,Niedostosowanie prędkości do warunków ruchu,nan,nan,0,0,1,0,50.239694,19.052333,5
98619,2017-04-29,04:45:00,19*03'095,50*14'229,KATOWICE,nan,nan,A4,"340,3",Obszar niezabudowany,...,Niedostosowanie prędkości do warunków ruchu,nan,nan,0,0,1,0,50.239694,19.052639,5
98527,2017-04-26,15:20:00,19*03'096,50*14'229,KATOWICE,GÓRNOŚLĄSKA,nan,G100446S,nan,Obszar niezabudowany,...,Niedostosowanie prędkości do warunków ruchu,nan,nan,0,0,1,0,50.239694,19.052667,5
112657,2019-05-15,16:00:00,19*03'088,50*14'230,KATOWICE,GÓRNOŚLĄSKA,nan,G100446S,nan,Obszar niezabudowany,...,Niedostosowanie prędkości do warunków ruchu,nan,nan,2,0,0,1,50.239722,19.052444,5
112628,2019-05-15,07:40:00,19*03'099,50*14'230,KATOWICE,GÓRNOŚLĄSKA,nan,G100446S,nan,Obszar niezabudowany,...,Nieprawidłowe zmienianie pasa ruchu,nan,nan,0,0,1,0,50.239722,19.05275,5
115854,2019-11-04,07:45:00,19*03'091,50*14'230,KATOWICE,GÓRNOŚLĄSKA,nan,G100446S,nan,Obszar niezabudowany,...,Niedostosowanie prędkości do warunków ruchu,nan,nan,0,0,1,0,50.239722,19.052528,5
100369,2017-08-19,23:20:00,19*03'096,50*14'230,KATOWICE,GÓRNOŚLĄSKA,nan,G100466S,nan,Obszar niezabudowany,...,Niedostosowanie prędkości do warunków ruchu,nan,nan,0,0,1,0,50.239722,19.052667,5
